In [ ]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL

In [ ]:
work_path=r'D:\university\研究生\tourism_forecasting_competition\submit' #####work_path must be adjusted before running the code.
os.chdir(work_path)

In [ ]:
country_name=["加拿大",'智利','墨西哥','台湾','香港','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','夏威夷','奥地利','捷克']

In [ ]:
flight=pd.read_excel(work_path+'\\'+'flight.xlsx')
flight=flight.replace({0:1})
flight.index=np.array(flight.iloc[:,0])
flight=flight.iloc[:,1:]
flight_2023=flight[flight.index.year==2023]
data=pd.read_excel(work_path+'\\'+'data.xlsx').iloc[:,1:]
data.columns=country_name
data.index=pd.date_range(start="1989-1", end="2023-02", freq="MS")
data_full=data.copy().iloc[:372,:]
data=data.iloc[360:]
fill1=[5,4,4,4,3,3,4,4,4,4,4,3,4,3,4,4,4,4,4,5]
fill2=[2,1,1,1,0,0,1,1,1,1,1,0,1,0,1,1,1,1,1,2]

In [ ]:
###using flight data to estimate reference series
flight_result=pd.DataFrame(np.zeros(shape=(6,20)),columns=country_name)
flight_result.index=pd.date_range(start="2023-01", end="2023-06", freq="MS")
flight_pre=list()
data_pre=list()
for i in range(20):
    flight_pre.append(flight.iloc[flight.shape[0]-fill1[i]-2,i])
    data_pre.append(data.iloc[data.shape[0]-fill2[i]-1,i])
for j in range(20):
    stl = STL(np.log(data_full.dropna().iloc[:,j]), period=12, seasonal=13)
    res = stl.fit()
    seasonal=pd.DataFrame(res.seasonal[:-1])
    q=pd.Series(seasonal.index).apply(lambda x:x.month)
    seasonal['month']=np.array(q)
    seasonal_mean=seasonal.groupby(by='month').mean()
    for i in range(6):
        if i<5-fill1[j]:
            flight_result.iloc[i,j]=data.iloc[48+i,j]
        else:
            flight_result.iloc[i,j]=(flight_2023.iloc[i,j]/flight_pre[j])*data_pre[j]*(1+seasonal_mean.iloc[i,0])
flight_result.to_excel(work_path+'\\reference_series'+'\\flight_result.xlsx')

In [ ]:
###Average the reference series estimated by search data and flight data
method_name=['holtwinters','ets','tbats','arimax_full','arimax_part','prophet_full','prophet_part']
reference=pd.DataFrame(index=np.arange(6),columns=country_name)
reference=reference.fillna(0)
for method in method_name:
    ref=pd.read_excel(work_path+'\\reference_series'+'\\'+method+'_result.xlsx')
    reference+=ref
reference=reference/7
flight_reference=pd.read_excel(work_path+'\\reference_series'+'\\'+'flight_result.xlsx').iloc[:,1:]
for country in ["加拿大",'台湾','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','奥地利']:
    reference[country]=(reference[country]+flight_reference[country])/2
reference.to_excel('reference.xlsx')